Input File Creation
===========
First let's start with some tools to create input files for a given deployment schedule.

In [1]:
import os
import sys
import uuid
import json
import time
import subprocess
from math import ceil
from copy import deepcopy

import numpy as np
import pandas as pd
import cymetric as cym
%matplotlib inline
import matplotlib.pyplot as plt
import george

import dtw

/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
with open('once-through.json') as f:
    BASE_SIM = json.load(f)
DURATION = BASE_SIM['simulation']['control']['duration']
YEARS = ceil(DURATION / 12)
MONTH_SHUFFLE = (1, 7, 10, 4, 8, 6, 12, 2, 5, 9, 11, 3)
NULL_SCHEDULE = {'build_times': [{'val': 1}], 
                 'n_build': [{'val': 0}], 
                 'prototypes': [{'val': 'LWR'}]}
LWR_PROTOTYPE = {'val': 'LWR'}
OPT_H5 = 'opt.h5'

In [3]:
BASE_SIM['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

In [4]:
def deploy_inst_schedule(Θ):
    if np.sum(Θ) == 0: 
        return NULL_SCHEDULE
    sched = {'build_times': {'val': []},
             'n_build': {'val': []},
             'prototypes': {'val': []}}
    build_times = sched['build_times']['val']
    n_build = sched['n_build']['val']
    prototypes = sched['prototypes']['val']
    m = 0
    for i, θ in enumerate(Θ):
        if θ <= 0:
            continue
        build_times.append(i*12 + MONTH_SHUFFLE[m])
        n_build.append(int(θ))
        prototypes.append('LWR')
        m = (m + 1) % 12
    return sched

def make_sim(Θ, fname='sim.json'):
    sim = deepcopy(BASE_SIM)
    inst = sim['simulation']['region']['institution']
    inst['config']['DeployInst'] = deploy_inst_schedule(Θ)
    with open(fname, 'w') as f:
        json.dump(sim, f)
    return sim

In [5]:
s = make_sim([])

In [6]:
s['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

Simulate
=========
Now let's build some tools to run simulations and extract a GWe time series.

In [7]:
def run(fname='sim.json', out=OPT_H5):
    """Runs a simulation and returns the sim id."""
    cmd = ['cyclus', '--warn-limit', '0', '-o', out, fname]
    proc = subprocess.run(cmd, check=True, universal_newlines=True, stdout=subprocess.PIPE)
    simid = proc.stdout.rsplit(None, 1)[-1]
    return simid

ZERO_GWE = pd.DataFrame({'GWe': np.zeros(YEARS)}, index=np.arange(YEARS))
ZERO_GWE.index.name = 'Time'

def extract_gwe(simid, out=OPT_H5):
    """Computes the annual GWe for a simulation."""
    with cym.dbopen(out) as db:
        evaler = cym.Evaluator(db)
        raw = evaler.eval('TimeSeriesPower', conds=[('SimId', '==', uuid.UUID(simid))])
    ano = pd.DataFrame({'Time': raw.Time.apply(lambda x: x//12), 
                        'GWe': raw.Value.apply(lambda x: 1e-3*x/12)})
    gwe = ano.groupby('Time').sum()
    gwe = (gwe + ZERO_GWE).fillna(0.0)
    return np.array(gwe.GWe)

Distancing
========
Now let's build some tools to distance between a GWe time series and a demand curve.

In [8]:
DEFAULT_DEMAND = 90 * (1.01**np.arange(YEARS))  # 1% growth

In [9]:
def d(g, f=None):
    """The dynamic time warping distance between a GWe time series and a demand curve."""
    f = DEFAULT_DEMAND if f is None else f
    rtn = dtw.distance(f[:, np.newaxis], g[:, np.newaxis])
    return rtn

def αd(g, f=None, dtol=1e-5):
    """Computes the mininmal α and the DTW d."""
    f = DEFAULT_DEMAND if f is None else f
    C = dtw.cost_matrix(f[:, np.newaxis], g[:, np.newaxis])
    d = dtw.distance(cost=C)
    #d_t = np.diagonal(C) / (np.arange(1, C.shape[0] + 1) + np.arange(1, C.shape[1] + 1))
    d_t = np.diagonal(C) / np.sum(C.shape)
    #α = np.argwhere(np.cumsum(d_t <= dtol) != np.arange(1, len(d_t) + 1))[0,0]
    filt = np.argwhere(d_t <= dtol)
    α = filt[-1,-1] if len(filt) > 0 else 0
    #α = filt[0,0] if len(filt) > 0 else 0
    print("Simulation α", α)
    print("Simulation d(t)", d_t)
    return α, d

def gwed(Θ, f=None, dtol=1e-5, find_α=False):
    """For a given deployment schedule Θ, return the GWe time series and the distance 
    to the demand function f.
    """
    make_sim(Θ)
    simid = run()
    gwe = extract_gwe(simid)
    if find_α:
        rtn = αd(gwe, f=f, dtol=dtol)
    else:
        rtn = d(gwe, f=f)
    return (gwe,) + rtn

Initialize Optimization
===============
Now let's start with a couple of simple simulations

In [10]:
N = np.asarray(np.ceil(4*(1.01)**np.arange(YEARS)), dtype=int)  # max annual deployments
Θs = [] # deployment schedules
G = []  # GWe per sim
D = []  # distances per sim
if os.path.isfile(OPT_H5):
    os.remove(OPT_H5)

In [11]:
def add_sim(Θ, f=None, dtol=1e-5):
    """Add a simulation to the known simulations by performing the simulation."""
    g_s, α, d_s = gwed(Θ, f=f, dtol=dtol, find_α=True)
    Θs.append(Θ)
    G.append(g_s)
    D.append(d_s)
    return α

First, add a schedule where nothing is deployed, leaving the initial facilities to retire.

In [12]:
add_sim(np.zeros(YEARS, dtype=int))

Simulation α 0
Simulation d(t) [  0.04833333   0.05016667   0.08492333   0.1488609    0.24540451
   0.37881355   0.54168169   0.73743684   0.97034121   1.23132462
   1.53131787   1.86041938   2.22372857   2.62467919   3.05337265
   3.51907804   4.01939882   4.55193948   5.11930554   5.72243693
   6.35810796   7.03142738   7.73583832   8.4764517    9.25337955
  10.06506835  10.91079903  11.79068702  12.70401556  13.65340238
  14.6389664   15.65916107  16.71660768  17.80892876  18.93708138
  20.10285719  21.30388243  22.54195125  23.8163591   25.12640269
  26.46638005  27.81975718  29.18666809  30.5672481   31.96163392
  33.36996359  34.79237656  36.22901366  37.68001713  39.14553063]


0

Next, add a simulation that is the max deployment schedule to bound the space

In [13]:
add_sim(N)

Simulation α 0
Simulation d(t) [  1.16666667e-02   7.10000000e-02   1.34576667e-01   2.36472433e-01
   3.23885215e-01   4.41944067e-01   5.36262039e-01   6.83066193e-01
   8.06287715e-01   9.15163374e-01   1.02139344e+00   1.16770073e+00
   1.32994840e+00   1.48369971e+00   1.62257310e+00   1.78794182e+00
   1.95230050e+00   2.13143123e+00   2.29098570e+00   2.49686978e+00
   2.70072297e+00   2.89689973e+00   3.08079894e+00   3.30466910e+00
   3.54732316e+00   3.78033684e+00   4.00910261e+00   4.28936438e+00
   4.56826787e+00   4.87839020e+00   5.16057347e+00   5.49516595e+00
   5.83787507e+00   6.15723134e+00   6.48226712e+00   6.87402745e+00
   7.29335939e+00   7.69237527e+00   8.08127345e+00   8.50360275e+00
   8.93014870e+00   9.40783104e+00   9.89381006e+00   1.04962490e+01
   1.11213573e+01   1.17619059e+01   1.24124975e+01   1.31568664e+01
   1.39810109e+01   1.48024031e+01]


0

Optimizer
=======
Now let's add some tools to do the estimation phase of the optimization.

In [14]:
Γ = 285
np.random.seed(424242)

In [15]:
def gp_gwe(Θs, G, α, T=None, tol=1e-6, verbose=False):
    """Create a Gaussian process regression model for GWe."""
    S = len(G)
    T = YEARS if T is None else T
    t = np.arange(T)
    P = len(Θs[0])
    ndim = P + 1 - α
    y_mean = np.mean(G)
    y = np.concatenate(G)
    x = np.empty((S*T, ndim), dtype=int)
    for i in range(S):
        x[i*T:(i+1)*T, 0] = t
        x[i*T:(i+1)*T, 1:] = Θs[i][np.newaxis, α:]
    yerr = tol * y_mean
    #kernel = float(y_mean) * george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    #for p in range(P):
    #    kernel *= george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    #kernel = float(y_mean) * george.kernels.Matern52Kernel(1.0, ndim=ndim)
    kernel = float(y_mean) * george.kernels.Matern32Kernel(1.0, ndim=ndim)
    gp = george.GP(kernel, mean=y_mean)
    gp.compute(x, yerr=yerr, sort=False)
    gp.optimize(x, y, yerr=yerr, sort=False, verbose=verbose)
    return gp, x, y

def predict_gwe(Θ, gp, y, α, T=None):
    """Predict GWe for a deployment schedule Θ and a GP."""
    T = YEARS if T is None else T
    t = np.arange(T)
    P = len(Θ)
    ndim = P + 1 - α
    x = np.empty((T, ndim), dtype=int)
    x[:,0] = t
    x[:,1:] = Θ[np.newaxis,α:]
    mu = gp.predict(y, x, mean_only=True)
    return mu

def gp_d_inv(θ_p, D_inv, tol=1e-6, verbose=False):
    """Computes a Gaussian process model for a deployment parameter."""
    S = len(D)
    ndim = 1
    x = θ_p
    y = D_inv
    y_mean = np.mean(y)
    yerr = tol * y_mean
    kernel = float(y_mean) * george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    gp = george.GP(kernel, mean=y_mean, solver=george.HODLRSolver)
    gp.compute(x, yerr=yerr, sort=False)
    gp.optimize(x, y, yerr=yerr, sort=False, verbose=verbose)
    return gp, x, y

def weights(Θs, D, N, Nlower, α, tol=1e-6, verbose=False):
    P = len(N)
    θ_ps = np.array(Θs)
    D = np.asarray(D)
    D_inv = D**-1 
    W = [None] * α
    for p in range(α, P):
        θ_p = θ_ps[:,p]
        range_p = np.arange(Nlower[p], N[p] + 1)
        gp, _, _ = gp_d_inv(θ_p, D_inv, tol=tol, verbose=verbose)
        d_inv_np = gp.predict(D_inv, range_p, mean_only=True)
        #p_min = np.argmin(D)
        #lam = θ_p[p_min]
        #fact = np.cumprod([1.0] + list(range(1, N[p] + 1)))[Nlower[p]:N[p] + 1]
        #d_inv_np = np.exp(-lam) * (lam**range_p) / fact
        if np.all(np.isnan(d_inv_np)) or np.all(d_inv_np <= 0.0):
            # try D, instead of D^-1
            #gp, _, _ = gp_d_inv(θ_p, D, tol=tol, verbose=verbose)
            #d_np = gp.predict(D, np.arange(0, N[p] + 1), mean_only=True)
            # try setting the shortest d to 1, all others 0.
            #d_inp_np = np.zeros(N[p] + 1, dtype='f8')
            #p_min = np.argmin(D)
            #d_inv_np[np.argwhere(θ_p[p_min] == range_p)] = 1.0
            # try Poisson dist centered at min.
            p_min = np.argmin(D)
            lam = θ_p[p_min]
            fact = np.cumprod([1.0] + list(range(1, N[p] + 1)))[Nlower[p]:N[p] + 1]
            d_inv_np = np.exp(-lam) * (lam**range_p) / fact
        if np.any(d_inv_np < 0.0):
            d_inv_np[d_inv_np < 0.0] = np.min(d_inv_np[d_inv_np > 0.0])
        d_inv_np_tot = d_inv_np.sum()
        w_p = d_inv_np / d_inv_np_tot
        W.append(w_p)
    return W

def guess_scheds(Θs, W, Γ, gp, y, α, T=None):
    """Guess a new deployment schedule, given a number of samples Γ, weights W, and 
    Guassian process for the GWe.
    """
    P = len(W)
    Θ_γs = np.empty((Γ, P), dtype=int)
    Θ_γs[:, :α] = Θs[0][:α]
    for p in range(α, P):
        w_p = W[p]
        Θ_γs[:, p] = np.random.choice(len(w_p), size=Γ, p=w_p)
    Δ = []
    for γ in range(Γ):
        Θ_γ = Θ_γs[γ]
        g_star = predict_gwe(Θ_γ, gp, y, α, T=T)
        d_star = d(g_star)
        Δ.append(d_star)
    γ = np.argmin(Δ)
    Θ_γ = Θ_γs[γ]
    print('hyperparameters', gp.kernel[:])
    #print('Θ_γs', Θ_γs)
    #print('Θ_γs[γ]', Θ_γs[γ])
    #print('Predition', Δ[γ], Δ)
    return Θ_γ, Δ[γ]

def guess_scheds_loop(Θs, gp, y, N, Nlower):
    """Guess a new deployment schedule, given a number of samples Γ, weights W, and 
    Guassian process for the GWe.
    """
    P = len(N)
    Θ = np.array(Θs[0], dtype=int)
    for p in range(P):
        d_p = []
        range_p = np.arange(Nlower[p], N[p] + 1, dtype=int)
        for n_p in range_p:
            Θ[p] = n_p
            g_star = predict_gwe(Θ, gp, y, α=0, T=p+1)[:p+1]
            d_star = d(g_star, f=DEFAULT_DEMAND[:p+1])
            d_p.append(d_star)
        Θ[p] = range_p[np.argmin(d_p)]
    print('hyperparameters', gp.kernel[:])
    return Θ, np.min(d_p)

def estimate(Θs, G, D, N, Nlower, Γ, α, T=None, tol=1e-6, verbose=False, method='stochastic'):
    """Runs an estimation step, returning a new deployment schedule."""
    gp, x, y = gp_gwe(Θs, G, α, T=T, tol=tol, verbose=verbose)
    if method == 'stochastic':
        # orig
        W = weights(Θs, D, N, Nlower, α, tol=tol, verbose=verbose)
        Θ, dmin = guess_scheds(Θs, W, Γ, gp, y, α, T=T)
    elif method == 'inner-prod':
        # inner prod
        Θ, dmin = guess_scheds_loop(Θs, gp, y, N, Nlower)
    elif method == 'all':
        W = weights(Θs, D, N, Nlower, α, tol=tol, verbose=verbose)
        Θ_stoch, dmin_stoch = guess_scheds(Θs, W, Γ, gp, y, α, T=T)
        Θ_inner, dmin_inner = guess_scheds_loop(Θs, gp, y, N, Nlower)
        if dmin_stoch < dmin_inner:
            winner = 'stochastic'
            Θ = Θ_stoch
        else:
            winner = 'inner'
            Θ = Θ_inner
        print('Estimate winner is {}'.format(winner))
    else:
        raise ValueError('method {} not known'.format(method))
    return Θ

def optimize(MAX_D=0.1, MAX_S=12, T=None, tol=1e-6, dtol=1e-5, verbose=False):
    global Θs, G, D
    α = 0
    s = 2
    z = 2
    n = N
    nlower = n0 = np.zeros(len(N), dtype=int)
    dtol = np.linspace(dtol * 2.0 / len(N), dtol, len(N))
    method = 'stochastic'
    #method = 'all'
    while MAX_D < D[-1] and s < MAX_S and α + 1 < YEARS:
        print(s)
        print('-'*18)
        Gprev = np.array(G[:z])
        t0 = time.time()
        method = 'stochastic' if s%4 < 2 else 'all'
        Θ = estimate(Θs, G, D, n, nlower, Γ, α, T=T, tol=tol, verbose=verbose, method=method)
        t1 = time.time()
        α_s = add_sim(Θ, dtol=dtol)
        t2 = time.time()
        print('Estimate time:   {0} min {1} sec'.format((t1-t0)//60, (t1-t0)%60))
        print('Simulation time: {0} min {1} sec'.format((t2-t1)//60, (t2-t1)%60))
        print(D)
        sys.stdout.flush()
        idx = [int(i) for i in np.argsort(D)[:z]]
        if D[-1] == max(D):
            idx.append(-1)
        #elif len(G) == z + 2  and np.allclose(G[:z], Gprev):
        #    n = np.array([Θs[0] + 1, N], dtype=int).min(axis=0)
        #    nlower = np.array([Θs[0] - 1, n0], dtype=int).max(axis=0)
        #    print('New N-upper', n)
        #    print('New N-lower', nlower)
        #if (α < α_s) and ((len(D) - 1) in idx[:2]):
        #if (α < α_s) and (len(D) == idx[0] + 1):
        if (len(D) == idx[0] + 1):
            print('Update α: {0} -> {1}'.format(α, α_s))
            α = α_s
        #    method = 'stochastic'
        #elif method == 'stochastic' and len(D) == z + 2:
        #elif len(D) == z + 2:
        #    print('Trying inner product method')
        #    method = 'inner-prod'
        #else:
        #    print('Trying stochastic method')
        #    method = 'stochastic'
        #elif α > 0:
        #    print('Update α: {0} -> {1}'.format(α, α - 1))
        #    α -= 1
        Θs = [Θs[i] for i in idx]
        G = [G[i] for i in idx]
        D = [D[i] for i in idx]
        s += 1
        print()

Simulate
=======
Ok! Let's try it.

In [16]:
%%time
optimize(MAX_S=25, dtol=1e-5)

2
------------------
hyperparameters [ 8.44155389  5.41756185]
hyperparameters [ 8.44155389  5.41756185]
Estimate winner is inner
Simulation α 0
Simulation d(t) [ 0.04833333  0.05016667  0.08492333  0.1488609   0.24540451  0.37881355
  0.54168169  0.73743684  0.97034121  1.23132462  1.53131787  1.86041938
  2.22372857  2.62467919  3.00753932  3.40657804  3.79856549  4.15610615
  4.52347221  4.85577026  5.20810796  5.51059404  5.80250498  6.10311837
  6.38587955  6.10855501  5.42453913  5.30496029  5.4124555   4.65216848
  4.55939917  4.24411294  4.36155955  3.99181954  3.94801078  3.94139337
  3.93777646  3.95849049  4.02824931  4.07816213  4.11669415  4.2323508
  4.3672495   4.57482883  4.77047313  5.07220136  5.42436825  5.77744383
  6.15926431  6.57002393]
Estimate time:   0.0 min 9.885438442230225 sec
Simulation time: 0.0 min 7.120560646057129 sec
[39.145530632616016, 14.802403115793279, 6.5700239322320666]
Update α: 0 -> 0

3
------------------
hyperparameters [ 7.8516871  4.63620

In [17]:
Θs[0]

array([0, 3, 3, 3, 0, 4, 2, 4, 3, 2, 2, 1, 1, 5, 5, 3, 5, 4, 2, 0, 4, 3, 4,
       3, 5, 2, 3, 3, 5, 4, 2, 5, 2, 4, 2, 4, 5, 6, 2, 2, 5, 2, 6, 1, 6, 2,
       5, 3, 3, 5])

In [18]:
G[0]

array([  85.16666667,   93.83333333,   92.33333333,   92.83333333,
         92.5       ,   92.16666667,   92.33333333,   93.41666667,
         91.91666667,   95.5       ,   92.75      ,   92.83333333,
         90.75      ,   92.        ,   96.83333333,   95.33333333,
         94.75      ,  100.16666667,   98.41666667,   98.25      ,
         97.83333333,   95.83333333,  101.91666667,   99.83333333,
        100.33333333,  102.16666667,  102.66666667,  105.        ,
        105.33333333,  104.08333333,  109.16666667,  106.33333333,
        109.25      ,  108.33333333,  109.66666667,  111.58333333,
        111.33333333,  112.16666667,  118.16666667,  117.25      ,
        117.5       ,  121.66666667,  125.08333333,  128.33333333,
        132.41666667,  133.66666667,  141.33333333,  143.41666667,
        143.58333333,  149.        ])

In [19]:
DEFAULT_DEMAND

array([  90.        ,   90.9       ,   91.809     ,   92.72709   ,
         93.6543609 ,   94.59090451,   95.53681355,   96.49218169,
         97.45710351,   98.43167454,   99.41599129,  100.4101512 ,
        101.41425271,  102.42839524,  103.45267919,  104.48720598,
        105.53207804,  106.58739882,  107.65327281,  108.72980554,
        109.8171036 ,  110.91527463,  112.02442738,  113.14467165,
        114.27611837,  115.41887955,  116.57306835,  117.73879903,
        118.91618702,  120.10534889,  121.30640238,  122.5194664 ,
        123.74466107,  124.98210768,  126.23192876,  127.49424804,
        128.76919052,  130.05688243,  131.35745125,  132.67102577,
        133.99773602,  135.33771338,  136.69109052,  138.05800142,
        139.43858144,  140.83296725,  142.24129692,  143.66370989,
        145.10034699,  146.55135046])

In [20]:
(G[0] - DEFAULT_DEMAND) / DEFAULT_DEMAND

array([-0.0537037 ,  0.03226989,  0.00571113,  0.00114576, -0.01232576,
       -0.02562866, -0.03353137, -0.0318732 , -0.05685001, -0.02978385,
       -0.0670515 , -0.07545868, -0.10515536, -0.10181157, -0.06398429,
       -0.08760759, -0.10216873, -0.06023913, -0.08579959, -0.09638393,
       -0.10912481, -0.13597714, -0.09022818, -0.11764883, -0.12200961,
       -0.11481842, -0.11929343, -0.10819542, -0.11422208, -0.13339968,
       -0.10007498, -0.1321107 , -0.11713363, -0.13320926, -0.13122878,
       -0.12479712, -0.13540395, -0.13755686, -0.10041901, -0.11623507,
       -0.12311951, -0.10101432, -0.08491963, -0.070439  , -0.05035848,
       -0.05088511, -0.00638326, -0.00171959, -0.01045493,  0.01670847])

In [21]:
sum(np.abs(G[0] - DEFAULT_DEMAND))

475.37818233899884

In [22]:
1.7 / 40

0.042499999999999996

In [23]:
Θs[1] - Θs[0]

array([ 0, -2, -3,  1,  0,  1,  2, -1,  1, -2,  3,  1,  3, -1, -3,  0,  0,
       -1,  3,  0,  0,  0,  0,  3,  0, -1, -1, -1,  0,  1,  0,  0,  0, -1,
        0,  0,  0,  0,  0, -1, -1,  0,  0,  3,  1, -1,  0, -1, -1,  0])

In [24]:
sum(N)

285